### Une approche purement supervisée

Lorsqu'il s'agit de résoudre un problème de classification supervisée, le choix du modèle approprié revêt une importance cruciale pour obtenir des performances optimales. 
Dans le cadre de notre étude sur la prédiction des tags, on a sélectionné quelques algorithmes de classification populaires, chacun avec ses propres caractéristiques et avantages.

- **Dummy Classifier** :
    Le Dummy Classifier est un modèle de base utilisé comme référence pour évaluer la performance des autres modèles. Il assigne les étiquettes de manière aléatoire ou selon une stratégie simple. Bien qu'il ne fournisse pas de prédictions précises, il est utile pour évaluer si nos modèles sont significativement meilleurs que la prédiction aléatoire.

- **SGD Classifier (Stochastic Gradient Descent)** :
    Le SGD Classifier est un modèle linéaire entraîné à l'aide de la descente de gradient stochastique. Il est efficace pour traiter de grands ensembles de données et est souvent utilisé dans des scénarios où le nombre de fonctionnalités est élevé. Il peut être utilisé pour une variété de tâches de classification et est particulièrement utile lorsque le temps de formation est un facteur critique.

- **Logistic Regression** :
    La régression logistique est un modèle linéaire utilisé pour la classification binaire et multiclasse. Malgré son nom, il est principalement utilisé pour la classification plutôt que pour la régression. Il fournit des probabilités de classe et est interprétable, ce qui le rend utile pour comprendre l'importance des fonctionnalités.

- **Multinomial Naive Bayes (MultinomialNB)** :
    Le modèle Naive Bayes multinomial est basé sur le théorème de Bayes avec une supposition naive d'indépendance entre les fonctionnalités. Il est couramment utilisé pour la classification de texte, en particulier pour les tâches où les fonctionnalités sont des comptages d'occurrences de mots.

- **Linear Support Vector Classifier (LinearSVC)** :
    Le LinearSVC est une implémentation de SVM (Support Vector Machine) linéaire, qui est un modèle de classification puissant. Il vise à trouver l'hyperplan qui sépare au mieux les classes dans l'espace des fonctionnalités. Le LinearSVC est efficace pour les ensembles de données de grande taille et à haute dimensionnalité.

- **Perceptro**n :
    Le Perceptron est un modèle de classification linéaire basé sur un neurone artificiel simple. Il est entraîné à l'aide de l'algorithme de descente de gradient stochastique et est adapté aux problèmes de classification binaire.

- **Passive Aggressive Classifier** :
    Le Passive Aggressive Classifier est un modèle de classification en ligne qui est souvent utilisé pour les tâches de classification de texte. Il est capable de s'adapter rapidement aux nouvelles données et est utile dans les scénarios où les données arrivent en continu.

Chacun de ces modèles présente des avantages et des limitations, on va explorer ces différents modèles pour déterminer celui qui offre les meilleures performances pour notre ensemble de données.

### Méthodes d’évaluation



- **Jaccard Score** :
    Le score Jaccard est une mesure de similarité entre deux ensembles. Dans le contexte de la classification multi-labels, il mesure la similarité entre les ensembles de vrais et de prédictions étiquetés. Il est calculé en divisant la taille de l'intersection des ensembles par la taille de leur union. 
  => Plus le score Jaccard est proche de 1, plus les ensembles sont similaires.

- **Hamming Loss** :
    La perte de Hamming mesure la fraction des étiquettes mal prédites par rapport au nombre total d'étiquettes. Elle est calculée en prenant la moyenne des erreurs de prédiction pour chaque échantillon. 
  => Une Hamming loss de 0 indique une prédiction parfaite, et une Hamming loss de 1 indique une prédiction complètement incorrecte.

- **Accuracy (Précision)** :
    L'exactitude mesure la proportion d'étiquettes correctement prédites par rapport au nombre total d'étiquettes. C'est une mesure globale de la performance du modèle. 
  => Une précision de 1 indique une prédiction parfaite, tandis qu'une précision de 0 indique une prédiction complètement incorrecte.

- **Precision (Précision)** :
    La précision mesure la proportion d'étiquettes prédites comme positives qui sont réellement positives. Elle est calculée en divisant le nombre de vrais positifs par la somme des vrais positifs et des faux positifs. 
  => Une précision élevée indique un faible taux de faux positifs.

- **Recall (Rappel)** :
    Le rappel mesure la proportion d'étiquettes positives réellement prédites par rapport à toutes les étiquettes positives. Il est calculé en divisant le nombre de vrais positifs par la somme des vrais positifs et des faux négatifs. 
  => Un rappel élevé indique un faible taux de faux négatifs.

Ces métriques d'évaluation sont importantes pour évaluer la performance des modèles de classification multi-labels. 

###### imports

In [5]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import SGDClassifier, LogisticRegression, Perceptron, PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import hamming_loss, accuracy_score, precision_score, recall_score, f1_score

In [6]:
import warnings

# Ignorer tous les avertissements
warnings.filterwarnings("ignore")


In [7]:
data = pd.read_csv(f'data_final.csv')

In [8]:
# Supprimer la colonne Unnamed: 0
data = data.drop(columns=['Unnamed: 0'])

In [9]:
data.head(3)

,Question,Tags,nb_tags,question_length
0,fastest way get value looking fastest way obta...,performance algorithm,2,197
1,use socket api issue getting socket api work p...,c++ c,2,106
2,net xml comment api documentation easy way pro...,visual-studio,1,78


In [10]:
data['Tags'] = data['Tags'].str.split()

In [11]:
# Prendre un échantillon aléatoire de 2000 lignes du DataFrame
sample_data = data.sample(n=2000, random_state=42)

In [12]:
X = sample_data['Question']
y = sample_data['Tags']

In [13]:
y.iloc[0]

['database', 'algorithm']

In [14]:
multilabel_binarizer = MultiLabelBinarizer()
y_bin = multilabel_binarizer.fit_transform(y)

In [15]:
y_bin[:2]

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0]])

In [16]:
[(index_label, yy) for (index_label, yy) in enumerate(y_bin[0]) if yy==1]

[(0, 1), (12, 1)]

In [17]:
classes = multilabel_binarizer.classes_
print(classes)
len(classes)

['algorithm' 'amazon-web-services' 'android' 'angular' 'arrays' 'asp.net'
 'asp.net-mvc' 'c' 'c#' 'c++' 'c++11' 'css' 'database' 'django' 'docker'
 'gcc' 'html' 'ios' 'iphone' 'java' 'javascript' 'jquery' 'json' 'linux'
 'macos' 'multithreading' 'mysql' 'node.js' 'objective-c' 'pandas'
 'performance' 'php' 'postgresql' 'python' 'r' 'reactjs' 'ruby'
 'ruby-on-rails' 'spring' 'spring-boot' 'sql' 'sql-server' 'string'
 'swift' 'typescript' 'unit-testing' 'visual-studio' 'windows' 'xcode']


49

In [18]:
vectorizer_X = TfidfVectorizer(analyzer = 'word',
                                       min_df=0.0,
                                       max_df = 1.0,
                                       strip_accents = None,
                                       encoding = 'utf-8', 
                                       preprocessor=None,
                                       token_pattern=r"(?u)\S\S+",
                                       max_features=1000)


In [19]:
X_tfidf = vectorizer_X.fit_transform(X)

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y_bin, test_size = 0.2, random_state = 0) # Do 80/20 split

In [21]:
y_test

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

Pour évaluer les modèles, on va utiliser le score Jaccard car il est le mieux adapté à la classification multi labels

In [22]:
def jacard_score(y_true,y_pred):
    jacard = np.minimum(y_true,y_pred).sum(axis=1) / np.maximum(y_true,y_pred).sum(axis=1)
    return jacard.mean()*100

In [23]:
def calculate_results(y_pred, y_test, model):
    jaccard = jacard_score(y_test, y_pred)
    hamming = hamming_loss(y_test, y_pred) 
    accuracy = accuracy_score(y_test, y_pred) 
    precision = precision_score(y_test, y_pred, average='samples')
    recall = recall_score(y_test, y_pred, average='samples') 
    f1 = f1_score(y_test, y_pred, average='macro')
    return jaccard, hamming, accuracy, precision, recall, f1


In [24]:
dummy = DummyClassifier()
sgd = SGDClassifier()
lr = LogisticRegression()
mn = MultinomialNB()
svc = LinearSVC()
perceptron = Perceptron()
pac = PassiveAggressiveClassifier()



In [25]:
# Définir les classificateurs à tester
classifiers = [DummyClassifier(), SGDClassifier(), LogisticRegression(), MultinomialNB(), 
               LinearSVC(), Perceptron(), PassiveAggressiveClassifier()]

# Initialiser une liste pour stocker les résultats
results_list = []

# Définir les noms des modèles
model_names = ['Dummy', 'SGD', 'Logistic Regression', 'Multinomial NB', 'Linear SVC', 'Perceptron', 'Passive Aggressive']

# Définir les méthodes d'évaluation
evaluation_methods = ['Jaccard score', 'Hamming loss', 'Accuracy', 'Precision', 'Recall', 'f1']

# Boucler sur chaque classificateur
for classifier, model_name in zip(classifiers, model_names):
    # Créer le modèle OneVsRestClassifier
    model = OneVsRestClassifier(classifier)
    
    # Entraîner le modèle
    model.fit(X_train, y_train)
    
    # Faire des prédictions
    y_pred = model.predict(X_test)
    
    # Calculer les résultats
    jaccard, hamming, accuracy, precision, recall, f1 = calculate_results(y_pred, y_test, model)
    
    # Ajouter les résultats à la liste
    results_list.append([model_name, jaccard, hamming, accuracy, precision, recall, f1])

# Convertir la liste en DataFrame
results_df = pd.DataFrame(results_list, columns=['Modèle'] + evaluation_methods)

# Transposer le DataFrame pour avoir les modèles en colonnes et les méthodes d'évaluation en lignes
results_df = results_df.set_index('Modèle').T


In [26]:
results_df

Modèle,Dummy,SGD,Logistic Regression,Multinomial NB,Linear SVC,Perceptron,Passive Aggressive
Jaccard score,0.000000,36.414881,10.254167,6.254167,34.212500,35.239881,38.023214
Hamming loss,0.034745,0.029745,0.032092,0.032908,0.026888,0.037551,0.031378
Accuracy,0.000000,0.200000,0.072500,0.035000,0.227500,0.182500,0.227500
Precision,0.000000,0.463333,0.136250,0.087917,0.443542,0.446000,0.479375
Recall,0.000000,0.434125,0.102667,0.067125,0.364958,0.449333,0.453292
f1,0.000000,0.398650,0.052032,0.051079,0.367560,0.368399,0.411051


In [27]:
model = OneVsRestClassifier(SGDClassifier())

# Entraîner le modèle
model.fit(X_train, y_train)

OneVsRestClassifier(estimator=SGDClassifier())

In [28]:
from joblib import dump, load

# Save the model
dump(model, 'model_sgd.joblib')
dump(vectorizer_X, 'vectorizer_X.joblib')
dump(multilabel_binarizer, 'multilabel_binarizer.joblib')

['multilabel_binarizer.joblib']

In [29]:
# Load the model
loaded_model = load('model_sgd.joblib')
loaded_vectorizer = load('vectorizer_X.joblib')


In [30]:
X_test.shape

(400, 1000)

Test

In [43]:
x = loaded_vectorizer.transform(["how to install api with python? is java that hard? can you help me with python"])
x

<1x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [44]:
predictions = loaded_model.predict(x)

In [45]:
predictions


array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0]])

In [46]:
predicted_labels = [classes[i] for i, prediction in enumerate(predictions[0]) if prediction == 1]

print("Predicted labels:", predicted_labels)


Predicted labels: ['java', 'python']
